In [1]:
# Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path

%matplotlib inline

In [1]:
public class Card{
private int rank, suit;

private String[] suitNames = new String[]{ "H", "C", "S", "D" };
private String[] rankNumber = new String[]{ "A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K" };
private int[] points = new int[]{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10};

Card(int suitIndex, int rankIndex){
    rank = rankIndex;
    suit = suitIndex;
}

public @Override String toString(){
    return rankNumber[rank]+suitNames[suit]; 
}

public int getRank(){
    return rank;
}

public int getSuit(){
    return suit;
}

public String getSuitName(){
    return suitNames[suit];
}

public String getRankName(){
    return rankNumber[rank];
}

public int getPoints(){
     return points[rank];
}

public ImageIcon ImageOfCard() throws Exception{
    ImageIcon icon = new ImageIcon("/StandardDeck/GameCards/"+getRankName() + getSuitName()+".png");
    return icon;
}
}

SyntaxError: invalid syntax (<ipython-input-1-f6b74d79b057>, line 1)

## Data Preprocessing

RNN input requires an array data type. The `full_review_text` column will be transformed into the `X` array and the “sentiment” column into the `y` array.

To train the RNN model, we need to encode the text data as an integer. This transformation can be done using the following tools from Keras.

In [4]:
# Import Keras modules for data encoding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
# Create an instance of the Tokenizer and fit it with the X text data
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(X)

In [8]:
# Padding sequences
X_pad = pad_sequences(X_seq, maxlen=140, padding="post")

Now that the data is encoded, the training and testing sets will be created.

In [9]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pad, y, random_state=78)

## Build and Train the LSTM RNN Model

In this section, a custom LSTM RNN model is going to be designed in Keras, and it's going to be fitted (trained) using the training data we defined.

These are the steps that will be followed:

* Define the model architecture in Keras.

* Compile the model.

* Fit the model to the training data.

In [10]:
# Import Keras modules for model creation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

 ### Making Predictions